Reference for Algorithm:
Olfati-Saber, R. (2006): Flocking for multi-agent dynamic systems: Algorithms and theory. IEEE Transactions on Automatic Control.
https://ieeexplore.ieee.org/document/1605401

In [ ]:
## Sqare Formation

In [1]:
# === IMPORTS AND DISPLAY SETUP ===
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button
from IPython.display import HTML

# === PARAMETERS ===
n_drones = 4
dt = 0.1
max_speed = 1.0
target = np.array([10.0, 10.0])
formation_offset = np.array([[-1, 0], [0, 1], [1, 0], [0, -1]])  # Square formation offsets
initial_center = np.array([0.0, 0.0])
initial_positions = initial_center + formation_offset

# === GRAPH STRUCTURE FOR LAPLACIAN CONSENSUS ===
A = np.array([
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]
], dtype=float)
D = np.diag(np.sum(A, axis=1))
L = D - A

# === OBSTACLE SETUP ===
obstacle = Polygon([[5, 5], [6, 7], [7, 6], [6, 5]], closed=True, edgecolor='r', facecolor='r', alpha=0.5)

# === HELPER FUNCTIONS ===
def point_to_segment_distance(point, seg_start, seg_end):
    seg_vec = seg_end - seg_start
    point_vec = point - seg_start
    seg_len = np.linalg.norm(seg_vec)
    proj = np.dot(point_vec, seg_vec) / seg_len
    proj = np.clip(proj, 0, seg_len)
    closest = seg_start + proj * seg_vec / seg_len
    return np.linalg.norm(point - closest)

def obstacle_repulsion(pos, obstacle):
    repulsion = np.zeros(2)
    for i in range(len(obstacle.xy) - 1):
        start, end = obstacle.xy[i], obstacle.xy[i + 1]
        dist = point_to_segment_distance(pos, start, end)
        if dist < 1.5:
            center = (start + end) / 2
            repulsion += (pos - center) / (dist + 1e-6) * (1.5 - dist) * 0.8
    return repulsion

def update_positions(positions, velocities):
    return positions + velocities * dt

# === FORMATION CONTROL LAW ===
def update_velocities(positions, target, L, formation_offset, max_speed):
    n = len(positions)
    centroid = np.mean(positions, axis=0)
    to_target = target - centroid
    dist = np.linalg.norm(to_target)
    group_velocity = to_target / (dist + 1e-6) * min(dist, max_speed)

    new_vel = np.zeros_like(positions)
    for i in range(n):
        desired_pos = centroid + formation_offset[i]
        offset_error = desired_pos - positions[i]
        consensus = -0.2 * sum(L[i, j] * (positions[i] - positions[j]) for j in range(n))
        repulsion = obstacle_repulsion(positions[i], obstacle)
        combined = group_velocity + 0.6 * offset_error + consensus + repulsion
        speed = np.linalg.norm(combined)
        new_vel[i] = combined / speed * max_speed if speed > max_speed else combined
    return new_vel

# === PLOTTING SETUP ===
fig, ax = plt.subplots()
ax.set_xlim(-5, 15)
ax.set_ylim(-5, 15)
ax.set_aspect('equal')
ax.add_patch(obstacle)

uav_plot, = ax.plot([], [], 'bo', markersize=8)
target_plot, = ax.plot([], [], 'ro', markersize=8)
trajectory_plots = [ax.plot([], [], 'b-', alpha=0.3)[0] for _ in range(n_drones)]

positions = initial_positions.copy()
trajectories = [np.array([p]) for p in positions]
animation_running = True

# === ANIMATION TOGGLE ===
def toggle_animation(event):
    global animation_running
    animation_running = not animation_running

# === MAIN UPDATE LOOP ===
def update(frame):
    global positions
    centroid = np.mean(positions, axis=0)
    if np.linalg.norm(centroid - target) < 0.5:
        return uav_plot, target_plot, *trajectory_plots
    if animation_running:
        velocities = update_velocities(positions, target, L, formation_offset, max_speed)
        positions[:] = update_positions(positions, velocities)

    uav_plot.set_data(positions[:, 0], positions[:, 1])
    target_plot.set_data([target[0]], [target[1]])
    for i in range(n_drones):
        trajectories[i] = np.vstack([trajectories[i], positions[i]])
        trajectory_plots[i].set_data(trajectories[i][:, 0], trajectories[i][:, 1])
    return uav_plot, target_plot, *trajectory_plots

# === BUTTON AND ANIMATION ===
ax_button = plt.axes([0.81, 0.01, 0.1, 0.075])
button = Button(ax_button, 'Pause/Play')
button.on_clicked(toggle_animation)
ani = FuncAnimation(fig, update, frames=400, interval=100, blit=True)
HTML(ani.to_jshtml())


Output hidden; open in https://colab.research.google.com to view.

In [2]:
## Rotating V Shape Formation

In [3]:
# === IMPORTS AND SETUP ===
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Circle
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button
from IPython.display import HTML

# === PARAMETERS ===
n_drones = 4
dt = 0.1
max_speed = 1.0
target = np.array([10.0, 10.0])

# Body-frame V-formation offsets (local frame)
body_offsets = np.array([[-1.5, -1], [-0.5, 0], [0.5, 0], [1.5, -1]])

# Initial heading and placement
initial_heading = np.array([1.0, 1.0]) / np.sqrt(2)
initial_center = np.array([0.0, 0.0])

# === FORMATION ROTATION FUNCTION ===
def rotate_offsets(offsets, direction):
    theta = np.arctan2(direction[1], direction[0])
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    return offsets @ rot.T

# Initialize drone positions based on rotated V-offsets
positions = initial_center + rotate_offsets(body_offsets, initial_heading)
trajectories = [np.array([p]) for p in positions]

# === OBSTACLE SETUP ===
poly_obstacle = Polygon([[5, 5], [6, 7], [7, 6], [6, 5]], closed=True, edgecolor='r', facecolor='r', alpha=0.5)
wall_obstacle = Polygon([[2, 6], [2.2, 6], [2.2, 12], [2, 12]], closed=True, edgecolor='b', facecolor='b', alpha=0.5)
circle_obstacle = Circle((7, 2), radius=1.0, edgecolor='g', facecolor='g', alpha=0.5)
obstacle_list = [poly_obstacle, wall_obstacle, circle_obstacle]

# === OBSTACLE REPULSION LOGIC ===
def point_to_segment_distance(point, seg_start, seg_end):
    seg_vec = seg_end - seg_start
    point_vec = point - seg_start
    seg_len = np.linalg.norm(seg_vec)
    proj = np.dot(point_vec, seg_vec) / seg_len
    proj = np.clip(proj, 0, seg_len)
    closest = seg_start + proj * seg_vec / seg_len
    return np.linalg.norm(point - closest)

def obstacle_repulsion(pos, obstacle):
    repulsion = np.zeros(2)
    if isinstance(obstacle, Polygon):
        vertices = obstacle.get_xy()
        for i in range(len(vertices) - 1):
            start, end = vertices[i], vertices[i + 1]
            dist = point_to_segment_distance(pos, start, end)
            if dist < 1.5:
                center = (start + end) / 2
                repulsion += (pos - center) / (dist + 1e-6) * (1.5 - dist) * 0.6
    elif isinstance(obstacle, Circle):
        center = np.array(obstacle.center)
        radius = obstacle.radius
        vec = pos - center
        dist = np.linalg.norm(vec)
        if dist < radius + 1.2:
            repulsion += vec / (dist + 1e-6) * (radius + 1.2 - dist) * 1.2
    return repulsion

# === POSITION & CONTROL LOGIC ===
def update_positions(positions, velocities):
    return positions + velocities * dt

def update_velocities(positions, target, body_offsets, max_speed):
    centroid = np.mean(positions, axis=0)
    to_target = target - centroid
    heading = to_target / (np.linalg.norm(to_target) + 1e-6)
    rotated_offsets = rotate_offsets(body_offsets, heading)
    desired_positions = centroid + rotated_offsets

    new_vel = np.zeros_like(positions)
    for i in range(n_drones):
        formation_error = desired_positions[i] - positions[i]
        repulsion = sum(obstacle_repulsion(positions[i], obs) for obs in obstacle_list)
        combined = heading * max_speed + 0.8 * formation_error + repulsion
        speed = np.linalg.norm(combined)
        new_vel[i] = combined / speed * max_speed if speed > max_speed else combined
    return new_vel

# === PLOTTING SETUP ===
fig, ax = plt.subplots()
ax.set_xlim(-5, 15)
ax.set_ylim(-5, 15)
ax.set_aspect('equal')
for obs in obstacle_list:
    ax.add_patch(obs)

uav_plot, = ax.plot([], [], 'bo', markersize=8)
target_plot, = ax.plot([], [], 'ro', markersize=8)
trajectory_plots = [ax.plot([], [], 'b-', alpha=0.3)[0] for _ in range(n_drones)]

animation_running = True
arrival_threshold = 0.5

# === ANIMATION TOGGLE ===
def toggle_animation(event):
    global animation_running
    animation_running = not animation_running

# === MAIN UPDATE FUNCTION ===
def update(frame):
    global positions
    centroid = np.mean(positions, axis=0)
    to_target = target - centroid
    if np.linalg.norm(to_target) < arrival_threshold:
        return uav_plot, target_plot, *trajectory_plots

    if animation_running:
        velocities = update_velocities(positions, target, body_offsets, max_speed)
        positions[:] = update_positions(positions, velocities)

    uav_plot.set_data(positions[:, 0], positions[:, 1])
    target_plot.set_data([target[0]], [target[1]])
    for i in range(n_drones):
        trajectories[i] = np.vstack([trajectories[i], positions[i]])
        trajectory_plots[i].set_data(trajectories[i][:, 0], trajectories[i][:, 1])
    return uav_plot, target_plot, *trajectory_plots

# === BUTTON AND ANIMATION ===
ax_button = plt.axes([0.81, 0.01, 0.1, 0.075])
button = Button(ax_button, 'Pause/Play')
button.on_clicked(toggle_animation)
ani = FuncAnimation(fig, update, frames=500, interval=100, blit=True)
HTML(ani.to_jshtml())


Output hidden; open in https://colab.research.google.com to view.